In [1]:
2+2

//hdfs dfs -mkdir -p /apps/hudi/lib
//hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar /apps/hudi/lib/hudi-spark-bundle.jar
//hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar /apps/hudi/lib/spark-avro.jar


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1611463294011_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: Int = 4


In [2]:
%%configure -f
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1611463294011_0004,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1611463294011_0004,spark,idle,Link,Link,✔


In [3]:
//Initialize a Spark Session for Hudi
// Copy on Write
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor


//Where to Store Your Hudi Table 
val outputBucket = "emr-demo-processed-250718677561-us-east-1"
// input data set 
val inputDataBucket = "emr-demo-raw-250718677561-us-east-1"

//Specify common DataSourceWriteOptions int a single hudiOption variable 
val hudiTableName = "indian_food_review_cow"
val hudiTableRecordKey = "foodid"
val hudiTablePath = "s3://"+outputBucket+"/createdataset/"+hudiTableName
val hudiTablePartitionColumn = "diet"
val hudiTablePrecombineKey = "timestamp"


// read data from s3 
val sourceData = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/indian_food_withID_full.csv")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())

sourceData.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
outputBucket: String = emr-demo-processed-250718677561-us-east-1
inputDataBucket: String = emr-demo-raw-250718677561-us-east-1
hudiTableName: String = indian_food_review_cow
hudiTableRecordKey: String = foodid
hudiTablePath: String = s3://emr-demo-processed-250718677561-us-east-1/createdataset/indian_food_review_cow
hudiTablePartitionColumn: String = diet
hudiTablePrecombineKey: String = timestamp
sourceData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [foodid: string, name: string ... 9 more fields]
root
 |-- foodid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: string (nullable = true)
 |-- cook_time: string (nullable = true)
 |-- flavor_pr

In [4]:
sourceData.select("foodid","name","diet","state","flavor_profile","course","state","timestamp").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+----------+-------------+--------------+-------+-------------+----------+
|foodid|          name|      diet|        state|flavor_profile| course|        state| timestamp|
+------+--------------+----------+-------------+--------------+-------+-------------+----------+
|     1|    Balu shahi|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1611463753|
|     2|        Boondi|vegetarian|    Rajasthan|         sweet|dessert|    Rajasthan|1611463753|
|     3|Gajar ka halwa|vegetarian|       Punjab|         sweet|dessert|       Punjab|1611463753|
|     4|        Ghevar|vegetarian|    Rajasthan|         sweet|dessert|    Rajasthan|1611463753|
|     5|   Gulab jamun|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1611463753|
|     6|        Imarti|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1611463753|
|     7|        Jalebi|vegetarian|Uttar Pradesh|         sweet|dessert|Uttar Pradesh|1611463753|
|     8|    Kaju katli|vegetar

In [5]:
// Set up our Hudi Data Source Options
val hudiOptions = Map[String,String](
    HoodieWriteConfig.TABLE_NAME -> hudiTableName,
    //for this detaset use COPY_ON_WRITE storage strategy other option us MERGE_ON_READ
    DataSourceWriteOptions.STORAGE_TYPE_OPT_KEY -> "COPY_ON_WRITE", 
    //next three options configure what Hudi should use as its record key 
    DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "foodid",
    DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> "diet",
    DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "timestamp",
    //for this data set, we specify that we want to sync metadata with Hive 
    DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true",
    DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName,
    DataSourceWriteOptions.HIVE_PARTITION_FIELDS_OPT_KEY -> "diet",
    DataSourceWriteOptions.HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY -> classOf[MultiPartKeysValueExtractor].getName
    )

//getName,
//    "hoodie.parquet.max.file.size" -> String.valueOf(1024 * 1024 * 1024),
//    "hoodie.parquet.small.file.limit" -> String.valueOf(64 * 1024 * 1024),
//    "hoodie.parquet.compression.ratio" -> String.valueOf(0.5),
//    "hoodie.insert.shuffle.parallelism" -> String.valueOf(2))



// Write input data to Hudi 

(sourceData.write
  .format("org.apache.hudi")
 // Opreation  Key tells Hudi whether this is an Insert Upsert or Bulk Insert operation 
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL)
  .options(hudiOptions)
  .mode(SaveMode.Overwrite)
  .save(hudiTablePath)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> timestamp, hoodie.datasource.hive_sync.partition_fields -> diet, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.datasource.hive_sync.table -> indian_food_review_cow, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> foodid, hoodie.table.name -> indian_food_review_cow, hoodie.datasource.write.storage.type -> COPY_ON_WRITE, hoodie.datasource.write.partitionpath.field -> diet)


In [6]:
// read Hudi data set from s3 

val readOptimzedHudiViewDF = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))


// take a look at our data ... 
readOptimzedHudiViewDF.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [18]:
spark.sql("""select foodid , state  from indian_food_ro_table where state='-1' limit 25 """ ).show(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+
|foodid|state|
+------+-----+
|     8|   -1|
|    10|   -1|
|    11|   -1|
|    13|   -1|
|    95|   -1|
|    97|   -1|
|    99|   -1|
|   110|   -1|
|   112|   -1|
|   116|   -1|
|   118|   -1|
|   129|   -1|
|   131|   -1|
|   145|   -1|
|   146|   -1|
|   150|   -1|
|   155|   -1|
|   157|   -1|
|   159|   -1|
|   162|   -1|
|   163|   -1|
|   165|   -1|
|   232|   -1|
|   249|   -1|
+------+-----+



In [7]:
readOptimzedHudiViewDF.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- foodid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: string (nullable = true)
 |-- cook_time: string (nullable = true)
 |-- flavor_profile: string (nullable = true)
 |-- course: string (nullable = true)
 |-- state: string (nullable = true)
 |-- region: string (nullable = true)
 |-- timestamp: long (nullable = true)



In [14]:
spark.sql("""select _hoodie_commit_time ,_hoodie_commit_seqno ,_hoodie_record_key , _hoodie_partition_path , _hoodie_file_name , foodid from indian_food_ro_table limit 20 """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|foodid|
+-------------------+--------------------+------------------+----------------------+--------------------+------+
|     20210124044941|  20210124044941_1_1|                 1|            vegetarian|e5a3eba9-336f-4a4...|     1|
|     20210124044941|  20210124044941_1_2|                 2|            vegetarian|e5a3eba9-336f-4a4...|     2|
|     20210124044941|  20210124044941_1_3|                 3|            vegetarian|e5a3eba9-336f-4a4...|     3|
|     20210124044941|  20210124044941_1_4|                 4|            vegetarian|e5a3eba9-336f-4a4...|     4|
|     20210124044941|  20210124044941_1_5|                 5|            vegetarian|e5a3eba9-336f-4a4...|     5|
|     20210124044941|  20210124044941_1_6|                 6|            vegetarian|e5a3eba9-336

In [9]:
// remove files from s3 raw bucket 

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

FileSystem.get(new URI("s3n://"+inputDataBucket), sc.hadoopConfiguration).delete(new Path("s3n://"+inputDataBucket+"/"), true)

//get Delta file - Nifi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
res40: Boolean = true


In [10]:
// read data from s3 - Delta data 
val sourceData_delta = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())

sourceData.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sourceData_delta: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [foodid: string, name: string ... 9 more fields]
root
 |-- foodid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: string (nullable = true)
 |-- cook_time: string (nullable = true)
 |-- flavor_profile: string (nullable = true)
 |-- course: string (nullable = true)
 |-- state: string (nullable = true)
 |-- region: string (nullable = true)
 |-- timestamp: long (nullable = false)



In [11]:
// take a look at our data ... 
sourceData_delta.registerTempTable("indian_food_delta");
spark.sql("""select state , count(*) from indian_food_delta group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+
| state|count(1)|
+------+--------+
|Punjab|      24|
+------+--------+



In [19]:
spark.sql("""select * from indian_food_delta """ ).show(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+--------------------+----------+---------+---------+--------------+-----------+------+------+----------+
|foodid|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile|     course| state|region| timestamp|
+------+--------------+--------------------+----------+---------+---------+--------------+-----------+------+------+----------+
|     8|    Kaju katli|Cashews, ghee, ca...|vegetarian|       10|       20|         sweet|    dessert|Punjab|    -1|1611463905|
|    10|         Kheer|Milk, rice, sugar...|vegetarian|       10|       40|         sweet|    dessert|Punjab|    -1|1611463905|
|    11|         Laddu|Gram flour, ghee,...|vegetarian|       10|       40|         sweet|    dessert|Punjab|    -1|1611463905|
|    13|     Nankhatai|Refined flour, be...|vegetarian|       20|       30|         sweet|    dessert|Punjab|    -1|1611463905|
|    95|       Khichdi|Moong dal, green ...|vegetarian|       40|       20|         spicy|main course|Pu

In [20]:
//Update 

// IMP - Before , if you wanted to update data in s3 ,, you have to read the old data , merge with the new data
// and then overwrite the old data .. with Hudi , we can directly update the data in-Place

(sourceData_delta.write
 .format("org.apache.hudi")
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
 .options(hudiOptions)
 .mode(SaveMode.Append)
 .save(hudiTablePath)
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
// read OptimizedHudiViewDF from updated data set 

val readOptimzedHudiViewDF_updated = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

readOptimzedHudiViewDF_updated.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF_updated: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      56|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
|      Telangana|       5|
+---------------+--------+
only showing top 20 rows



In [22]:
// remove files from s3 raw bucket 

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

FileSystem.get(new URI("s3n://"+inputDataBucket), sc.hadoopConfiguration).delete(new Path("s3n://"+inputDataBucket+"/"), true)

//get Delta file for delete ops - Nifi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
res70: Boolean = true


In [23]:
// read data from s3 - Delta data 
val sourceData_delta_delete = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())

sourceData.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sourceData_delta_delete: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [foodid: string, name: string ... 9 more fields]
root
 |-- foodid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: string (nullable = true)
 |-- cook_time: string (nullable = true)
 |-- flavor_profile: string (nullable = true)
 |-- course: string (nullable = true)
 |-- state: string (nullable = true)
 |-- region: string (nullable = true)
 |-- timestamp: long (nullable = false)



In [24]:
// take a look at our data ... 
sourceData_delta_delete.registerTempTable("indian_food_delta");
spark.sql("""select * from indian_food_delta  """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+--------------------+--------------+---------+---------+--------------+-----------+-----+------+----------+
|foodid|    name|         ingredients|          diet|prep_time|cook_time|flavor_profile|     course|state|region| timestamp|
+------+--------+--------------------+--------------+---------+---------+--------------+-----------+-----+------+----------+
|   212|Vindaloo|Chicken, coconut ...|non_vegetarian|       10|       40|         spicy|main course|  Goa|  West|1611464602|
|   252| Bebinca|Coconut milk, egg...|    vegetarian|       20|       60|         sweet|    dessert|  Goa|  West|1611464602|
|   255|  Pinaca|Brown rice, fenne...|    vegetarian|       -1|       -1|         sweet|    dessert|  Goa|  West|1611464602|
+------+--------+--------------------+--------------+---------+---------+--------------+-----------+-----+------+----------+



In [25]:
(sourceData_delta_delete.write
 .format("org.apache.hudi")
 .options(hudiOptions)
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
 .option(DataSourceWriteOptions.PAYLOAD_CLASS_OPT_KEY, "org.apache.hudi.common.model.EmptyHoodieRecordPayload")
 .mode(SaveMode.Append)
 .save(hudiTablePath))


//You can also hard delete data by setting OPERATION_OPT_KEY to DELETE_OPERATION_OPT_VAL to
//remove all records in the dataset you submit. For instructions on performing soft deletes, 
//and for more information about deleting data stored in Hudi tables, see Deletes in the Apache Hudi documentation.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
// read OptimizedHudiViewDF from updated data set 

val readOptimzedHudiViewDF_postdelete = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

readOptimzedHudiViewDF_postdelete.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF_postdelete: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      56|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
|      Telangana|       5|
|        Tripura|       1|
+---------------+--------+
only showing top 20 rows



In [27]:
// Point in time SQL 


val commits = (spark.sql(""" select distinct(_hoodie_commit_time) as commitTime from 
indian_food_ro_table order by commitTime""").map(k=> k.getString(0)).take(50))

commits.toString()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commits: Array[String] = Array(20210124044941, 20210124045902)
res95: String = [Ljava.lang.String;@4a5bf4c4


In [28]:
import org.apache.hudi.DataSourceReadOptions
val beginTime = "0"

val endTime = commits(0)

val dataSet = (spark.read
     .format("org.apache.hudi")
     // Mark that we want to do an incremental query 
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginTime) 
     .option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)  
     .options(hudiOptions)
     .load(hudiTablePath)) 

dataSet.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.hudi.DataSourceReadOptions
beginTime: String = 0
endTime: String = 20210124044941
dataSet: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [29]:
val beginTime = commits(0)

val endTime = commits(1)

val dataSet1 = (spark.read
     .format("org.apache.hudi")
     // Mark that we want to do an incremental query 
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginTime) 
     .option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)  
     .options(hudiOptions)
     .load(hudiTablePath)) 

dataSet1.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

beginTime: String = 20210124044941
endTime: String = 20210124045902
dataSet1: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+------+--------+
| state|count(1)|
+------+--------+
|Punjab|      24|
+------+--------+



In [32]:
val beginTime = commits(1)

//val endTime = commits(1)

val dataSet2 = (spark.read
     .format("org.apache.hudi")
     // Mark that we want to do an incremental query 
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginTime) 
     //.option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)  
     .options(hudiOptions)
     .load(hudiTablePath)) 

dataSet2.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

beginTime: String = 20210124045902
dataSet2: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+-----+--------+
|state|count(1)|
+-----+--------+
+-----+--------+



In [35]:
// remove files from s3 raw bucket 

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

FileSystem.get(new URI("s3n://"+inputDataBucket), sc.hadoopConfiguration).delete(new Path("s3n://"+inputDataBucket+"/"), true)

//get full file for  - Nifi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
res127: Boolean = true


In [36]:
//Using MoR - Hudi

//Where to Store Your Hudi Table 
val outputBucket = "emr-demo-processed-250718677561-us-east-1"
// input data set 
val inputDataBucket = "emr-demo-raw-250718677561-us-east-1"

val hudiTableName = "indian_food_review_mor"
val hudiTableRecordKey = "foodid"
val hudiTablePath = "s3://"+outputBucket+"/createdataset/"+hudiTableName
val hudiTablePartitionColumn = "diet"
val hudiTablePrecombineKey = "timestamp"


val hudiOptions = Map[String,String](
    HoodieWriteConfig.TABLE_NAME -> hudiTableName,
    //for this detaset use COPY_ON_WRITE storage strategy other option us MERGE_ON_READ
    DataSourceWriteOptions.STORAGE_TYPE_OPT_KEY -> "MERGE_ON_READ", 
    //next three options configure what Hudi should use as its record key 
    DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "foodid",
    DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> "diet",
    DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "timestamp",
    //for this data set, we specify that we want to sync metadata with Hive 
    DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true",
    DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName,
    DataSourceWriteOptions.HIVE_PARTITION_FIELDS_OPT_KEY -> "diet",
    DataSourceWriteOptions.HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY -> classOf[MultiPartKeysValueExtractor].getName
    )

// read data from s3 
val sourceData = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/*")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())

//sourceData.printSchema()

sourceData.select("foodid","name","diet","state","flavor_profile","course","state","timestamp").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

outputBucket: String = emr-demo-processed-250718677561-us-east-1
inputDataBucket: String = emr-demo-raw-250718677561-us-east-1
hudiTableName: String = indian_food_review_mor
hudiTableRecordKey: String = foodid
hudiTablePath: String = s3://emr-demo-processed-250718677561-us-east-1/createdataset/indian_food_review_mor
hudiTablePartitionColumn: String = diet
hudiTablePrecombineKey: String = timestamp
hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> timestamp, hoodie.datasource.hive_sync.partition_fields -> diet, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.datasource.hive_sync.table -> indian_food_review_mor, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> foodid, hoodie.table.name -> indian_food_review_mor, hoodie.datasource.write.storage.type -> MERGE_ON_READ, hoodie.datasource.write.partitionpath.field -> diet)
sourceDat

In [37]:
// Write input data to Hudi 

(sourceData.write
  .format("org.apache.hudi")
 // Opreation  Key tells Hudi whether this is an Insert Upsert or Bulk Insert operation 
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL)
  .options(hudiOptions)
  .mode(SaveMode.Overwrite)
  .save(hudiTablePath)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
spark.sql("SHOW TABLES").collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[default,indian_food_review_cow,false]
[default,indian_food_review_mor_ro,false]
[default,indian_food_review_mor_rt,false]
[,indian_food_delta,true]
[,indian_food_ro_table,true]


In [40]:
spark.sql("select * from indian_food_review_mor_ro").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+--------------------+---------+---------+--------------+-------+-----------+------+----------+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|foodid|          name|         ingredients|prep_time|cook_time|flavor_profile| course|      state|region| timestamp|      diet|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+--------------------+---------+---------+--------------+-------+-----------+------+----------+----------+
|     20210124051536| 20210124051536_1_54|                 1|            vegetarian|4ce369f6-33b0-41c...|     1|    Balu shahi|Maida flour, yogu...|       45|       25|         sweet|dessert|West Bengal|  East|1611463753|vegetarian|
|     20210124051536| 20210124051536_1_55|                 2|       

In [41]:
spark.sql("""select state , count(*) from indian_food_review_mor_ro group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [42]:
spark.sql("""select state , count(*) from indian_food_review_mor_rt group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [44]:
// read Hudi data set from s3 

val readOptimzedHudiViewDF = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

// take a look at our data ... 
readOptimzedHudiViewDF.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [46]:
// remove files from s3 raw bucket 

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

FileSystem.get(new URI("s3n://"+inputDataBucket), sc.hadoopConfiguration).delete(new Path("s3n://"+inputDataBucket+"/"), true)

//get full file for  - Nifi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.net.URI
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
res173: Boolean = true


In [47]:
// read data from s3 
val sourceData_delta = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/*")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())

//sourceData.printSchema()

sourceData_delta.select("foodid","name","diet","state","flavor_profile","course","state","timestamp").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sourceData_delta: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [foodid: string, name: string ... 9 more fields]
+------+--------------+----------+------+--------------+-----------+------+----------+
|foodid|          name|      diet| state|flavor_profile|     course| state| timestamp|
+------+--------------+----------+------+--------------+-----------+------+----------+
|     8|    Kaju katli|vegetarian|Punjab|         sweet|    dessert|Punjab|1611465599|
|    10|         Kheer|vegetarian|Punjab|         sweet|    dessert|Punjab|1611465599|
|    11|         Laddu|vegetarian|Punjab|         sweet|    dessert|Punjab|1611465599|
|    13|     Nankhatai|vegetarian|Punjab|         sweet|    dessert|Punjab|1611465599|
|    95|       Khichdi|vegetarian|Punjab|         spicy|main course|Punjab|1611465599|
|    97| Kulfi falooda|vegetarian|Punjab|         sweet|    dessert|Punjab|1611465599|
|    99|Lauki ki subji|vegetarian|Punjab|         spicy|main course|Punjab|1611465599|
|   11

In [48]:
(sourceData_delta.write
 .format("org.apache.hudi")
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
 .options(hudiOptions)
 .mode(SaveMode.Append)
 .save(hudiTablePath)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
// read OptimizedHudiViewDF from updated data set 

val readOptimzedHudiViewDF_updated = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

readOptimzedHudiViewDF_updated.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF_updated: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      56|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
|      Telangana|       5|
+---------------+--------+
only showing top 20 rows



In [51]:
spark.sql("""select state , count(*) from indian_food_review_mor_ro group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      56|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
|      Telangana|       5|
+---------------+--------+
only showing top 20 rows

